# Call Singular from GAP

Show the example from the [Computeralgebra-Rundrief Nr. 55, Oktober 2014](http://www.fachgruppe-computeralgebra.de/rundbrief/).

First load the GAP package.

In [1]:
LoadPackage( "JuliaExperimental", false );;

## Part 1:  Just call the appropriate Julia functions from GAP.

Define the ring with the default monomial ordering.
The syntax looks complicated; the reason is that the Julia syntax
for specifying the monomial ordering ('named arguments')
is not available in the low level interface.
(As a workaround, we can provide some Julia code, see Part 2 below.)

In [10]:
indetnames:= ConvertedToJulia( [ "x0", "x1", "x2", "x3" ] );
indetnames:= Julia.Base.convert( JuliaEvalString( "Array{String,1}" ),
                 indetnames );
Rinfo:= Julia.Singular.PolynomialRing( Julia.Singular.QQ, indetnames );
R:= Rinfo[1];
indets:= ConvertedFromJulia( Rinfo[2] );
x0:= indets[1];;  x1:= indets[2];;  x2:= indets[3];;  x3:= indets[4];;

<Julia: Any["x0", "x1", "x2", "x3"]>

<Julia: String["x0", "x1", "x2", "x3"]>

<Julia: (Singular Polynomial Ring (QQ),(x0,x1,x2,x3),(dp(4),C), Singular.spoly{Singular.n_Q}[x0, x1, x2, x3])>

<Julia: Singular Polynomial Ring (QQ),(x0,x1,x2,x3),(dp(4),C)>

[ <Julia: x0>, <Julia: x1>, <Julia: x2>, <Julia: x3> ]

Define the polynomial $( x_1 + x_3 )^2$.

In [11]:
p:= (x1+x3)^2;

<Julia: x1^2+2*x1*x3+x3^2>

Define the ideal in R generated by $x_0^2 - x_1 x_3$ and $x_0 x_1 - x_2 x_3$.

In [12]:
I:= Julia.Singular.Ideal( R, x0^2 - x1*x3, x0*x1 - x2*x3 );

<Julia: Singular Ideal over Singular Polynomial Ring (QQ),(x0,x1,x2,x3),(dp(4),C) with generators (x0^2-x1*x3, x0*x1-x2*x3)>

The corresponding matrix $i$ (of ideal generators).

In [13]:
i:= JuliaFunction( "Matrix", "Singular" )( I );

<Julia: [x0^2-x1*x3, x0*x1-x2*x3]>

The sum $I + I$ means the sum of ideals.

In [14]:
J:= I + I;

<Julia: Singular Ideal over Singular Polynomial Ring (QQ),(x0,x1,x2,x3),(dp(4),C) with generators (x0^2-x1*x3, x0*x1-x2*x3)>

Whereas $i + i$ means the sum of Singular matrices.

In [15]:
i + i;

<Julia: [2*x0^2-2*x1*x3, 2*x0*x1-2*x2*x3]>

The squared ideal $I^2$ inside $R$:

In [16]:
I2 := I^2;

<Julia: Singular Ideal over Singular Polynomial Ring (QQ),(x0,x1,x2,x3),(dp(4),C) with generators (x0^4-2*x0^2*x1*x3+x1^2*x3^2, x0^3*x1-x0*x1^2*x3-x0^2*x2*x3+x1*x2*x3^2, x0^2*x1^2-2*x0*x1*x2*x3+x2^2*x3^2)>

The Gröbner basis of the ideal $I$
is returned as a new different (but mathematically equal) ideal $G$.

In [17]:
G:= Julia.Base.std( I );

<Julia: Singular Ideal over Singular Polynomial Ring (QQ),(x0,x1,x2,x3),(dp(4),C) with generators (x0*x1-x2*x3, x0^2-x1*x3, x1^2*x3-x0*x2*x3)>

The syzygies of the generators of $G$ are the columns of a Singular module.

In [18]:
S:= Julia.Singular.syz( G );

<Julia: Singular Module over Singular Polynomial Ring (QQ),(x0,x1,x2,x3),(dp(4),C), with Generators:x0*gen(1)-x1*gen(2)-gen(3)x1*x3*gen(1)-x2*x3*gen(2)-x0*gen(3)>

To access the second column of $S$ use:

In [19]:
S[2];

<Julia: x1*x3*gen(1)-x2*x3*gen(2)-x0*gen(3)>

To create a matrix in `Singular.jl`, use the following.
```
Singular.matrix( R, 3, 2, [ x0, x3, x1, x2, x3, x0 ] )'
```
Via the GAP interface, the following works.
(We have to adjust the Julia type of the array,
in order to make it acceptable for `AbstractAlgebra.matrix`.)

In [22]:
arr1:= ConvertedToJulia( [ x0, x3, x1, x2, x3, x0 ] );;
arr2:= Julia.Base.convert(
JuliaEvalString( "Array{Singular.spoly{Singular.n_Q},1}" ),
arr1 );;
m:= Julia.AbstractAlgebra.matrix( R, 3, 2, arr2 );

<Julia: [x0 x3][x1 x2][x3 x0]>

To extract the $(2,1)$-entry from the matrix use the following.
(Access to 'rows' or 'columns' of such a matrix is not supported.)

In [23]:
m[ 2, 1 ];

<Julia: x1>

## Part 2:  Use some GAP structures on top of the Julia interface.

Define the ring with the monomial ordering `"degrevlex"`.

In [25]:
R:= SingularPolynomialRing( Rationals, [ "x0", "x1", "x2", "x3" ] :
        ordering:= "dp" );
AssignGeneratorVariables( R );

#I  Global variable `x0' is already defined and will be overwritten
#I  Global variable `x1' is already defined and will be overwritten
#I  Global variable `x2' is already defined and will be overwritten
#I  Global variable `x3' is already defined and will be overwritten
#I  Assigned the global variables [ x0, x1, x2, x3 ]


Singular_QQ[x0,x1,x2,x3]

Define the polynomial $( x_1 + x_3 )^2$.

In [27]:
p:= (x1+x3)^2;
IsSingularPolynomial( p );

<<Julia: x1^2+2*x1*x3+x3^2>>

true

Define the ideal in $R$ generated by $x_0^2 - x_1 x_3$ and $x_0 x_1 - x_2 x_3$.

In [28]:
I:= Ideal( R, [ x0^2 - x1*x3, x0*x1 - x2*x3 ] );

<semigroup>

Do not mix up the ideal and its list of generators.

In [29]:
i:= GeneratorsOfIdeal( I );

[ <<Julia: x0^2-x1*x3>>, <<Julia: x0*x1-x2*x3>> ]

The Gröbner basis of the ideal $I$
is returned as a new different (but mathematically equal) ideal $G$.

In [31]:
G:= GroebnerBasisIdeal( I );
GeneratorsOfIdeal( G );

<semigroup>

[ <<Julia: x0*x1-x2*x3>>, <<Julia: x0^2-x1*x3>>, <<Julia: x1^2*x3-x0*x2*x3>> ]